# **CASE: INVESTIMENTO EM AÇÕES**

---

### Instalação de bibliotecas

In [ ]:
# Para cálculo do VIF
install.packages("rms")
# Para cálculo do KS e AUC
install.packages("ROCR")

### Carregamento de bibliotecas

In [ ]:
library(rms)
library(ROCR)

### Leitura da base de dados

In [ ]:
dados_investimento <- read.table(file = "Investimento_Acoes.txt",
                                 sep = "\t",
                                 dec = ".",
                                 header = TRUE)

### Visualização da base de dados

In [ ]:
View(dados_investimento)

### Análise exploratória breve

In [ ]:
# Medidas resumo
summary(dados_investimento)

### Análise bivariada: variável resposta vs. variáveis explicativas

In [ ]:
# Comando auxiliar para aumentar resolução do gráfico (recomendado apenas no Colab)
reset_options <- options()
options(repr.plot.width = 20, repr.plot.height = 7)

In [ ]:
# Gráficos de boxplots
par(mfrow = c(1,5), cex = 1)
boxplot(Investimento_Fixa ~ Investiu_Variavel_6M,     data = dados_investimento, col = "darkturquoise", main = "Investimento em Renda Fixa")
boxplot(Investimento_Tesouro ~ Investiu_Variavel_6M,  data = dados_investimento, col = "darkturquoise", main = "Investimento em Tesouro")
boxplot(Investimento_Poupanca ~ Investiu_Variavel_6M, data = dados_investimento, col = "darkturquoise", main = "Investimento em Poupança")
boxplot(Rendimento_Liq_12M ~ Investiu_Variavel_6M,    data = dados_investimento, col = "darkturquoise", main = "Rendimento Líquido em 12M")
boxplot(Saldo_Conta ~ Investiu_Variavel_6M,           data = dados_investimento, col = "darkturquoise", main = "Saldo em Conta")

In [ ]:
# Resetando parâmetros gráficos (recomendado apenas no Colab)
options(reset_options)

### Modelo de regressão logística múltipla

In [ ]:
# Comando auxiliar para para omitir notação científica nos p-valores e controlar largura dos outputs na tela do Colab
options(scipen = 999, width = 200)

In [ ]:
# Ajuste do modelo 1: inicial
regressao_1 <- glm(Investiu_Variavel_6M ~
                     Investimento_Fixa +
                     Investimento_Tesouro +
                     Investimento_Poupanca +
                     Rendimento_Liq_12M +
                     Saldo_Conta,
                   family = binomial(link = 'logit'),
                   data = dados_investimento)

summary(regressao_1)

In [ ]:
# Ajuste do modelo 2: retirando Investimento_Tesouro
regressao_2 <- glm(Investiu_Variavel_6M ~
                     Investimento_Fixa +
                     Investimento_Poupanca +
                     Rendimento_Liq_12M +
                     Saldo_Conta,
                   family = binomial(link = 'logit'),
                   data = dados_investimento)

summary(regressao_2)

In [ ]:
# Avaliação de colinearidade no modelo 2
data.frame(VIF = vif(regressao_2))

In [ ]:
# Ajuste do modelo 3: retirando Rendimento_Liq_12M (colinearidade)
regressao_3 <- glm(Investiu_Variavel_6M ~
                     Investimento_Fixa +
                     Investimento_Poupanca +
                     Saldo_Conta,
                   family = binomial(link = 'logit'),
                   data = dados_investimento)

summary(regressao_3)

In [ ]:
# Avaliação de colinearidade no modelo 3
data.frame(VIF = vif(regressao_3))

### Aplicação do modelo 2 e definição de resposta predita

In [ ]:
# Aplicação do modelo na base (criação de uma nova coluna chamada "probabilidade")
dados_investimento$probabilidade_modelo_2 <- predict(regressao_2,
                                                     dados_investimento,
                                                     type = "response")

In [ ]:
# Definição de ponto de corte (padrão: % de 1's na amostra)
ponto_corte <- mean(dados_investimento$Investiu_Variavel_6M)
ponto_corte

In [ ]:
# Definição da resposta predita pelo modelo (criação de uma nova coluna chamada "predito")
dados_investimento$predito_modelo_2 <- as.factor(ifelse(dados_investimento$probabilidade_modelo_2 > ponto_corte, 1, 0))

### Análise de desempenho: modelo 2

In [ ]:
# Tabela de classificação
tabela <- table(dados_investimento$Investiu_Variavel_6M, dados_investimento$predito_modelo_2)
tabela

In [ ]:
# Acurácia
(tabela[1,1] + tabela[2,2]) / sum(tabela)

In [ ]:
# Especificidade
tabela[1,1] / (tabela[1,1] + tabela[1,2])

In [ ]:
# Sensibilidade
tabela[2,2] / (tabela[2,1] + tabela[2,2])

In [ ]:
# KS
pred <- prediction(dados_investimento$probabilidade_modelo_2, dados_investimento$Investiu_Variavel_6M)
perf <- performance(pred, "tpr", "fpr")
ks <- max(attr(perf, 'y.values')[[1]] - attr(perf, 'x.values')[[1]])

print(ks)

In [ ]:
# AUC
pred <- prediction(dados_investimento$probabilidade_modelo_2, dados_investimento$Investiu_Variavel_6M)
auc <- performance(pred, "auc")
auc <- auc@y.values[[1]]

print(auc)

### Aplicação do modelo 3 e definição de resposta predita

In [ ]:
# Aplicação do modelo na base (criação de uma nova coluna chamada "probabilidade")
dados_investimento$probabilidade_modelo_3 <- predict(regressao_3,
                                                     dados_investimento,
                                                     type = "response")

In [ ]:
# Definição de ponto de corte (padrão: % de 1's na amostra)
ponto_corte <- mean(dados_investimento$Investiu_Variavel_6M)
ponto_corte

In [ ]:
# Definição da resposta predita pelo modelo (criação de uma nova coluna chamada "predito")
dados_investimento$predito_modelo_3 <- as.factor(ifelse(dados_investimento$probabilidade_modelo_3 > ponto_corte, 1, 0))

### Análise de desempenho: modelo 3

In [ ]:
# Tabela de classificação
tabela <- table(dados_investimento$Investiu_Variavel_6M, dados_investimento$predito_modelo_3)
tabela

In [ ]:
# Acurácia
(tabela[1,1] + tabela[2,2]) / sum(tabela)

In [ ]:
# Especificidade
tabela[1,1] / (tabela[1,1] + tabela[1,2])

In [ ]:
# Sensibilidade
tabela[2,2] / (tabela[2,1] + tabela[2,2])

In [ ]:
# KS
pred <- prediction(dados_investimento$probabilidade_modelo_3, dados_investimento$Investiu_Variavel_6M)
perf <- performance(pred, "tpr", "fpr")
ks <- max(attr(perf, 'y.values')[[1]] - attr(perf, 'x.values')[[1]])

print(ks)

In [ ]:
# AUC
pred <- prediction(dados_investimento$probabilidade_modelo_3, dados_investimento$Investiu_Variavel_6M)
auc <- performance(pred, "auc")
auc <- auc@y.values[[1]]

print(auc)

### Intervalos de confiança

In [ ]:
# Escolha do beta (0 para intercepto, ou 1, 2, ... para os parâmetros referentes a cada variável explicativa)
beta = 0

In [ ]:
# Cálculo do intervalo
print("Limite inferior")
as.numeric(regressao_3$coefficients[beta + 1] - 1.96 * coef(summary(regressao_3))[beta + 1, "Std. Error"])
print("Limite superior")
as.numeric(regressao_3$coefficients[beta + 1] + 1.96 * coef(summary(regressao_3))[beta + 1, "Std. Error"])

### Exemplo de uso (aplicação) do modelo

In [ ]:
# Criação de base de dados com um novo investidor, que tem 6.000 reais investidos em renda fixa, não tem investimento em poupança, e tem 3.000 reais de saldo em conta
# Obs.: os nomes das colunas e padrão de conteúdo devem ser idênticos aos da base utilizada para construção do modelo
novos_dados <- data.frame(Investimento_Fixa     = c(6000),
                          Investimento_Poupanca = c(0),
                          Saldo_Conta           = c(3000))

In [ ]:
# Aplicação do modelo
novos_dados$PROB_INVESTIR_VARIAVEL <- predict(regressao_3, novos_dados, type = "response")
View(novos_dados)